In [ ]:
# Install llama-cpp-python.
!pip install llama-cpp-python

In [ ]:
# Import guidance.
import guidance

In [ ]:
# Create a LlamaCppSettings instance and set the model path.
settings = guidance.llms.LlamaCppSettings()
settings.model = "../ggml-v2-models/gpt4-x-vicuna-13B.ggml.q5_0.bin"

In [ ]:
# Create a LlamaCpp instance and pass the settings to it.
llama = guidance.llms.LlamaCpp(settings=settings)